This notebook seeks to explore the gender diversity of the different apache projects & the process

In [88]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import explode
from pyspark.sql.types import *

import json

In [2]:
session = SparkSession.builder.appName("whatCanWeLearnFromTheSixties").getOrCreate()
sc = session.sparkContext

The first thing we want to get is the committers and PMC members, this information is stored in LDAP but also available in JSON. Eventually we will want to enrich this with mailing list information

In [105]:
def loadFlatJsonFile(path, explodeKey, schema=None):
    """Load a flat multi-line json file and convert into Spark & explode"""
    rdd = sc.wholeTextFiles(path).values()
    df = (session.read.schema(schema)
            .json(rdd))
    return df.select(explode(explodeKey))

In [106]:
apache_people_schema = StructType([StructField("lastCreateTimestamp", StringType()),
                     StructField("people",
                                 MapType(StringType(), MapType(StringType(), StringType())))])
apache_people_df = loadFlatJsonFile(path="http_data_sources/public_ldap_people.json", # http://people.apache.org/public/public_ldap_people.json
                                 explodeKey="people", schema=apache_people_schema)

In [107]:
apache_people_df.take(10)

[Row(key='a_budroni', value={'name': 'Alessandro Budroni'}),
 Row(key='a_horuzhenko', value={'name': 'Artyom Horuzhenko'}),
 Row(key='aa', value={'name': 'Andrew Ahn'}),
 Row(key='aadamchik', value={'name': 'Andrus Adamchik'}),
 Row(key='aadomowski', value={'name': 'Aleksander Adamowski'}),
 Row(key='aadrian', value={'name': 'Adrian Antal'}),
 Row(key='aaf', value={'name': 'Alexei Fedotov', 'urls': '["http://www.dataved.ru/"]'}),
 Row(key='aajisaka', value={'name': 'Akira Ajisaka'}),
 Row(key='aalmiray', value={'name': 'Andres Almiray'}),
 Row(key='aanderson', value={'name': 'Aaron Anderson'})]